# Why the single-cavity equations do not work?

In [1]:
using CSV
using DataFrames
using Dates
using Expokit
using LaTeXStrings
using LinearAlgebra
using MKL
using Plots
using ProgressBars
using Revise
using SparseArrays
using TOML
# Saving the output matrix
using Serialization
using NPZ


includet("./modules/OpticalCavity.jl")
includet("./modules/Thermodynamics.jl")
includet("./modules/MasterEquations.jl")
includet("./modules/Phaseonium.jl")
includet("./modules/BosonicOperators.jl")
includet("./modules/Measurements.jl")


using .OpticalCavity
using .Thermodynamics
using .MasterEquations
using .Phaseonium
using .BosonicOperators
using .Measurements


includet("src/RoutineFunctions.jl")
includet("src/configurations_generation.jl") 

┌ Warning: Replacing docs for `BosonicOperators :: Union{}` in module `Main.BosonicOperators`
└ @ Base.Docs docs/Docs.jl:243
┌ Warning: Replacing docs for `Measurements :: Union{}` in module `Main.Measurements`
└ @ Base.Docs docs/Docs.jl:243
┌ Warning: Replacing docs for `Main.Measurements.temperature :: Tuple{Any, Any, Vararg{Any}}` in module `Main.Measurements`
└ @ Base.Docs docs/Docs.jl:243
┌ Warning: Replacing docs for `Main.Measurements.entropy_vn :: Tuple{Any, Vararg{Any}}` in module `Main.Measurements`
└ @ Base.Docs docs/Docs.jl:243
┌ Warning: Replacing docs for `Main.Measurements.avg_number :: Tuple{Any, Any, Vararg{Any}}` in module `Main.Measurements`
└ @ Base.Docs docs/Docs.jl:243
┌ Warning: Replacing docs for `Main.Measurements.avg_E :: Tuple{Any, Any, Vararg{Any}}` in module `Main.Measurements`
└ @ Base.Docs docs/Docs.jl:243


In [ ]:
CSV_FILE = "./simulations/simulations_ledger.csv";

csv = CSV.read(CSV_FILE, DataFrame, header=1);
dirs = ["simulations/simulation_$n" for n in csv[!, "meta_name"]]

# Remove all simulations and records on the csv after a certain index
# for dir in dirs[130:end]
#     rm(dir; force=true, recursive=true)
# end

In [ ]:
files_created, configurations = generate_configurations(config_file="fast_config.toml");

includet("src/main-iterator.jl")

main_iterator() 

remove_files = false
if remove_files
    # Remove report.txt from folder to make the simulation run again
    for dir in files_created
        isfile(dir * "/report.txt") && rm(dir * "/report.txt")
    end
end

In [2]:
function _check(ρ)
    println("System after the stroke:")
    println(checkdensity(ρ))
    println("Final Temperature of the System: $(Measurements.temperature(ρ, ω))")
end

function plot_stroke_state(system_evolution, cavity_evolution, α0)
    temperatures = []
    entropies = []
    for (i, ρ) in enumerate(system_evolution)
        local ω = α0 / cavity_evolution[i]
        t = Measurements.temperature(ρ, ω)
        ent = Measurements.entropy_vn(ρ)
        
        push!(temperatures, t)
        push!(entropies, ent)
    end

    g = plot(entropies, temperatures, label="Stroke")
    # Plot starting point
    scatter!(g, [entropies[1]], [temperatures[1]], label="Start", mc="blue", ms=5)
    # Plot ending point
    scatter!(g, [entropies[end]], [temperatures[end]], label="End", mc="red", ms=5)
    title!("Phaseonium Stroke (Isochoric)")
    xlabel!("Entropy")
    ylabel!("Temperature")
    display(g)
end


function plot_temperature(system_evolution, cavity_evolution, α0)    
    temperatures = []
    for (i, ρ) in enumerate(system_evolution)
        local ω = α0 / cavity_evolution[i]
        t = Measurements.temperature(ρ, ω)
        
        push!(temperatures, t)
    end

    g = plot(temperatures, label="Temperature")
    title!("Temperature Evolution")
    xlabel!("Time")
    ylabel!("Temperature")

    return g
end

function cascade_evolution(thermalizationtime, ρt, α0, bosonic_operators, ga, gb)
    # system_evolution, cavity_evolution = Thermodynamics.adiabatic_stroke(
    #     ρt, thermalizationtime, Δt, [a, ad], cavity; sampling_each=10)
    system_evolution = Thermodynamics.phaseonium_stroke_2(
        ρt, thermalizationtime, bosonic_operators, ga, gb; sampling_steps=50, verbose=3)

    return system_evolution
end


function onesystem_evolution(thermalizationtime, ρt, α0, bosonic_operators, ga, gb)
    # system_evolution, cavity_evolution = Thermodynamics.adiabatic_stroke(
    #     ρt, thermalizationtime, Δt, [a, ad], cavity; sampling_each=10)
    system_evolution = Thermodynamics.phaseonium_stroke(
        ρt, thermalizationtime, bosonic_operators, [ga, gb]; sampling_steps=100, verbose=3)

    return system_evolution
end

onesystem_evolution (generic function with 1 method)

# Thermalization

In [3]:
ndims = 250
Ω = 1.0
Δt = 1e-2

T_initial = 0.01
T_final = 0.08

# Create a Cavity
α0 = 2*π*100
surface = 50_000.0
l_min = 100_000.0
l_max = 101_000.0
acceleration = 0.0
expanding_force = 1e-12
compressing_force = 1e-12
cavity = Cavity(1.0, surface, l_min, l_max, α0, expanding_force, compressing_force)
ω = α0 / l_min
println("Cavity frequency: $ω")

global ρt = thermalstate(ndims, ω, T_initial)

# Create Phaseonium atoms
ϕ = π/1.8
α = Phaseonium.alpha_from_temperature(T_final, ϕ, ω)
println("Phaseonium α: $α < $(sqrt((1+cos(ϕ))/(3+cos(ϕ))))")

η = Phaseonium.densitymatrix(α, ϕ)

ga, gb = Phaseonium.dissipationrates(α, ϕ)
final_temperature = Phaseonium.finaltemperature(ω, ga, gb)
println(
    "Apparent Temperature carried by Phaseonium atoms: 
    $(final_temperature)")

# Define Kraus Operators
identity_op = I(ndims)
a = BosonicOperators.destroy(ndims)
ad = BosonicOperators.create(ndims)

C = BosonicOperators.C(Ω*Δt, ndims)
Cp = BosonicOperators.Cp(Ω*Δt, ndims)
S = BosonicOperators.S(Ω*Δt, ndims)
Sd = BosonicOperators.Sd(Ω*Δt, ndims)

E0 = sqrt(1 - ga/2 - gb/2) * identity(ndims)
E1 = sqrt(ga/2) * C
E2 = sqrt(ga) * S
E3 = sqrt(gb/2) * Cp
E4 = sqrt(gb) * Sd

kraus = [E0, E1, E2, E3, E4]
jumps = (a, ad)
ρt

Cavity frequency: 0.006283185307179587
Phaseonium α: 0.5257314910394288 < 0.5407162030609368
Apparent Temperature carried by Phaseonium atoms: 
    0.07999999999999984


250×250 Diagonal{Float64, Vector{Float64}}:
 0.466512   ⋅         ⋅         ⋅         …   ⋅            ⋅ 
  ⋅        0.248879   ⋅         ⋅             ⋅            ⋅ 
  ⋅         ⋅        0.132774   ⋅             ⋅            ⋅ 
  ⋅         ⋅         ⋅        0.0708332      ⋅            ⋅ 
  ⋅         ⋅         ⋅         ⋅             ⋅            ⋅ 
  ⋅         ⋅         ⋅         ⋅         …   ⋅            ⋅ 
  ⋅         ⋅         ⋅         ⋅             ⋅            ⋅ 
  ⋅         ⋅         ⋅         ⋅             ⋅            ⋅ 
  ⋅         ⋅         ⋅         ⋅             ⋅            ⋅ 
  ⋅         ⋅         ⋅         ⋅             ⋅            ⋅ 
 ⋮                                        ⋱               
  ⋅         ⋅         ⋅         ⋅             ⋅            ⋅ 
  ⋅         ⋅         ⋅         ⋅             ⋅            ⋅ 
  ⋅         ⋅         ⋅         ⋅             ⋅            ⋅ 
  ⋅         ⋅         ⋅         ⋅             ⋅            ⋅ 
  ⋅         ⋅         ⋅      

In [ ]:
function thermalization_stroke(systems, thermalizationtime)
    global ρt
    if systems == 1
        println("One system evolution")
        system_evolution = onesystem_evolution(thermalizationtime, ρt, α0, [C, Cp, S, Sd], ga, gb)
        figname = "phaseonium_stroke.png"
    else
        println("Cascade evolution")
        ρt = Matrix(kron(ρt, ρt))
        system_evolution = cascade_evolution(thermalizationtime, ρt, α0, [C, Cp, S, Sd], ga, gb)
        figname = "phaseonium_stroke_cascade.png"
    end

    cavity_evolution = [cavity.length for _ in 1:length(system_evolution)]

    g = plot_temperature(system_evolution, cavity_evolution, α0)
    times = range(1, length(system_evolution))
    γ = ga / gb
    # Thermalization function
    f(x, γ) = (final_temperature - T_initial) * (1 - exp(-γ * (x - 1))) + T_initial
    #plot!(g, times, f.(times, γ), label="Theoretical Temperature", lc=:red)
    display(g)
    # savefig(g, figname)



    return system_evolution
    
end

system_evolution = [];

In [ ]:
system_evolution = thermalization_stroke(1, 1_500_000)
system_evolution[end]

In [ ]:
system_evolution[end]

In [ ]:
compare_with = thermalstate(ndims, ω, T_final)

# Adiabatic Expansion

In [4]:
# global ρ = complex(copy(system_evolution[end]))
ρ = complex(thermalstate(ndims, ω, T_final))
n = ad * a
identity_matrix = spdiagm(ones(ndims))

cache = (;
        U = spzeros(ComplexF64, ndims, ndims),
        Ud = spzeros(ComplexF64, ndims, ndims),
        temp = zeros(ComplexF64, ndims, ndims),
        temp2 = zeros(ComplexF64, ndims, ndims),
        idd = identity_matrix,
        n = n,
        π_a = a * a,
        π_ad = ad * ad,
        π_op = Matrix{ComplexF64}(undef, ndims, ndims),
        _h = sparse(n .+ 0.5 .* identity_matrix),
        h1 = spzeros(ComplexF64, ndims, ndims),
        a1 = 0,
        p1 = 0,
        force = cavity.expanding_force,
    )

(U = sparse(Int64[], Int64[], ComplexF64[], 250, 250), Ud = sparse(Int64[], Int64[], ComplexF64[], 250, 250), temp = ComplexF64[0.0 + 0.0im 0.0 + 0.0im … 0.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 0.0 + 0.0im … 0.0 + 0.0im 0.0 + 0.0im; … ; 0.0 + 0.0im 0.0 + 0.0im … 0.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 0.0 + 0.0im … 0.0 + 0.0im 0.0 + 0.0im], temp2 = ComplexF64[0.0 + 0.0im 0.0 + 0.0im … 0.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 0.0 + 0.0im … 0.0 + 0.0im 0.0 + 0.0im; … ; 0.0 + 0.0im 0.0 + 0.0im … 0.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 0.0 + 0.0im … 0.0 + 0.0im 0.0 + 0.0im], idd = sparse([1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  241, 242, 243, 244, 245, 246, 247, 248, 249, 250], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  241, 242, 243, 244, 245, 246, 247, 248, 249, 250], [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0  …  1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], 250, 250), n = [0.0 0.0 … 0.0 0.0; 0.0 1.0 … 0.0 0.0; … ; 0.0 0.0 … 248.0 0.0; 0.0 0.0 … 0.0 248.99999999999997], π_a = [0.0 0.0 … 0.0 0.0; 0.0 0

In [5]:
function adiabatic_stroke(ρ, cavity, jumps, cache, sampling_steps)
    process = "Expansion"

    force = cavity.expanding_force

    # Initialize system tracking
    systems = Vector{Matrix{ComplexF64}}(undef, sampling_steps)
    systems[1] = ρ

    # Setup cavity
    cavity.acceleration = 0  # starts blocked

    # Determine expansion/contraction direction
    l_start = cavity.l_min
    l_end   = cavity.l_max
    direction = 1
    l_samplings = collect(range(l_start, stop=l_end, length=sampling_steps))
    
    cavity_lengths = [cavity.length for _ in 1:sampling_steps]

    iter = ProgressBar(total=sampling_steps)
    
    t = 0.0
    i = 2
    stop = false
    println("Starting!")
    while i <= sampling_steps
        if direction * cavity.length >= direction * l_samplings[i]
            systems[i] = ρ
            cavity_lengths[i] = cavity.length
            update(iter)
            i += 1
        elseif i > sampling_steps
            stop = true
        end
        ρ, cavity = MasterEquations.adiabaticevolve_1(
            ρ, cavity, cache, Δt, t, process, stop
            )
            t += Δt
                    
        if t % 25*Δt == 0
            println("force: $force, acceleration: $(cavity.acceleration)")
        end
    end
    
end

adiabatic_stroke (generic function with 1 method)

In [ ]:
adiabatic_stroke(ρ, cavity, jumps, cache, 1000)

In [ ]:
cavity.length

In [ ]:
function testprint()
    println("This is a test")
end

testprint()
testprint()